### Environment Setup and Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import threading
import os
from tqdm.auto import tqdm

#TF Logging is excessive, minimize log spamming by setting level lower
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed (only errors)
# 3 = INFO, WARNING, and ERROR messages are not printed

# On Mac you may encounter an error related to OMP, this is a workaround, but slows down the code
# https://github.com/dmlc/xgboost/issues/1715
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [ ]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Check if there is any available GPU

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=15360)]
    )


In [ ]:
from openbot import utils, train

## Hyperparameters

You may have to tune the learning rate and batch size depending on your available compute resources and dataset. As a general rule of thumb, if you increase the batch size by a factor of n, you can increase the learning rate by a factor of sqrt(n). In order to accelerate training and make it more smooth, you should increase the batch size as much as possible. For our working model, we used a batch size of 128. For debugging and hyperparamter tuning, you can set the number of epochs to a small value like 10. If you want to train a model which will achieve good performance, you should set it to 200 or more. For our working model, we used 600.

In [ ]:
params = train.Hyperparameters()
params.MODEL = "simple_cnn"
#Training Parameters
params.TRAIN_BATCH_SIZE = 128
params.TEST_BATCH_SIZE = 128
params.LEARNING_RATE = 0.01
params.NUM_EPOCHS = 10    #total number of iterations on the full dataset

#Data Augmentation Parameters
params.BATCH_NORM = False  #normalize the batched data (bad results)
params.TRANS_AUG = True    #translate the batched data (good results)
params.FLIP_AUG = True     #flip the images and steering angles (safe now for turns)
params.USE_LAST = False    #Use last trained model

#Callback Parameters
params.WANDB = False       #Use WANDB to log training metrics
params.CHKPT = True        #Save checkpoints at end of every epoch (slows down training)
params.LOG = True         #Save logs
params.TENSORBOARD = False #Use tensorboard to save data

## Load the dataset

In [ ]:
# Now we initiliaze a training object to hold these parameters, set the dataset directories of this object to point to the collected data
tr = train.Training(params, None)

tr.train_data_dir = "Dataset/train/tfrecords/train.tfrec"
tr.test_data_dir = "Dataset/test/tfrecords/train.tfrec"
train.load_tfrecord(tr, verbose=1)

Sanity check on the batch sampling, make sure the images and the steering/throttle look correct! -1 steering is the maximum left turn and 1.0 is the minimum right turn

In [ ]:
#TODO: get rid of the print statements for this
image_batch, label_batch = next(iter(tr.train_ds))
utils.show_train_batch(image_batch.numpy(), label_batch.numpy())

## Training

In [ ]:
# Create broadcast event so we can see the model fit processing
def broadcast(event, payload=None):
    print(event, payload)

event = threading.Event()
my_callback = train.MyCallback(broadcast, event)

model, callback_list = train.setup_training(tr, my_callback, "myModelName", verbose=True)

### Begin Training

In [ ]:
tr.history = model.fit(
    tr.train_ds,
    epochs=tr.hyperparameters.NUM_EPOCHS,
    use_multiprocessing=True,
    validation_data=tr.test_ds,
    verbose=1,
    callbacks=callback_list
)

## Evaluation

The loss and mean absolute error should decrease. This indicates that the model is fitting the data well. The custom metrics (direction and angle) should go towards 1. These provide some additional insight to the training progress. The direction metric measures weather or not predictions are in the same direction as the labels. Similarly the angle metric measures if the prediction is within a small angle of the labels. The intuition is that driving in the right direction with the correct steering angle is most critical part for good final performance.

### Convert to Tflite

In [ ]:
model_name = "myModelName.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()
open(model_name, "wb").write(tflite_model)

### Loss vs. Validation Loss

In [ ]:
loss = tr.history.history['loss']
val_loss = tr.history.history['val_loss']

In [ ]:
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="lower right")
plt.savefig("loss.png")

### Mean Absolute Error vs. Validation Mean Absolute Error

In [ ]:
mean_absolute_error = tr.history.history['mean_absolute_error']
val_mean_absolute_error = tr.history.history['val_mean_absolute_error']

In [ ]:
plt.plot(mean_absolute_error, label="mean_absolute_error")
plt.plot(val_mean_absolute_error, label="val_mean_absolute_error")
plt.xlabel("Epoch")
plt.ylabel("Mean Absolute Error")
plt.legend(loc="lower right")
plt.savefig("error.png")

### Direction Metric vs. Validation Direction Metric

In [ ]:
direction_metric = tr.history.history['direction_metric']
val_direction_metric = tr.history.history['val_direction_metric']

In [ ]:
plt.plot(direction_metric, label="direction_metric")
plt.plot(val_direction_metric, label="val_direction_metric")
plt.xlabel("Epoch")
plt.ylabel("Direction Metric")
plt.legend(loc="lower right")
plt.savefig("direction.png")

### Angle Metric vs. Validation Angle Metric

In [ ]:
angle_metric = tr.history.history['angle_metric']
val_angle_metric = tr.history.history['val_angle_metric']

In [ ]:
plt.plot(angle_metric, label="angle_metric")
plt.plot(val_angle_metric, label="val_angle_metric")
plt.xlabel("Epoch")
plt.ylabel("Angle Metric")  
plt.legend(loc="lower right")
plt.savefig("angle.png")